In [1]:
import pandas as pd
import numpy as np
import torch 
import torch.nn as  nn
from torch.utils.data import TensorDataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.preprocessing import StandardScaler
import time
from tqdm.autonotebook import tqdm, trange
import matplotlib.pyplot as plt

import PatchTST_models as tst 
device = 'cuda' if torch.cuda.is_available() else 'cpu'

#######################Тут не все компании!!!######################################
#######################Тут не все компании!!!######################################
# companies = ['SBER','GAZP' , 'LKOH', 'GMKN', 'MGNT',
#        'TATN', 'NVTK', 'SNGS', 'PLZL', 'SNGSP',
#        'PIKK', 'ROSN', 'SBERP', 'CHMF', 'NLMK',
#        'IRAO', 'YNDX', 'ALRS', 'RUAL', 'MTSS',
#        'MAGN', 'PHOR', 'RTKM', 'VTBR', 'TATNP',
#        'TCSG', 'AGRO', 'AFLT', 'AFKS',
#        'TRNFP', 'CBOM',
#        'FIVE', 'MOEX',
#        'HYDR']

companies = ['SBER' , 'SBERP', 'MOEX','VTBR']
######################Тут не все компании!!!##############################################
######################Тут не все компании!!!##############################################

/tmp/ipykernel_14743/3639851738.py:9: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


<p style="align: center;"><img src="https://s1.hostingkartinok.com/uploads/images/2024/01/6c6c3b261fd84cb562d2361f7ce47683.jpg", width=550, height=350></p>


In [2]:
df = pd.read_csv('~/Рабочий стол/ДИПЛОМ НОУТБУКИ/DL forcasitng diplom/Данные Мосбиржа/Total_MOEX_with_time_df')

In [3]:
max_len = 94740
data = np.zeros((max_len, len(companies)))
for i, company in enumerate(companies):
    series = np.array(df[df.secid == company].close)[:max_len]
    data[:,i] += series

In [4]:
data.shape

(94740, 4)

In [5]:
train_data = data[ : round(data.shape[0]*0.55) ]
val_data = data[ round(data.shape[0]*0.85) : ]

In [6]:
train_data, val_data = tst.all_data_scaler(train_data, val_data)

In [7]:
x_train, y_train = tst.sample_creator(train_data, 104)
x_val, y_val = tst.sample_creator(val_data, 104)

In [8]:
print(f'x_train: {x_train.shape}, y_train: {y_train.shape}\nx_val: {x_val.shape}, y_val: {y_val.shape}')

x_train: torch.Size([52003, 104, 4]), y_train: torch.Size([52003, 4])
x_val: torch.Size([14107, 104, 4]), y_val: torch.Size([14107, 4])


In [9]:
train_dataset = TensorDataset(x_train, y_train)
val_dataset  = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=150, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=150, shuffle=False)

In [10]:
print(tst.PatchTST.__doc__)


    Входыне данные:
    1) Кол-во рядов т.е. фичей: (M)
    2) Длина ряда: (L)
    3) Длина патча: (P)
    4) Страйд: (S)
    5) Кол-во головок памяти: (num_heads)
    6) Внутреннаяя размерность FFN: (dim_feedforward)
    7) Кол-во слоёв в энкодер блоке: (num_layers)
    8) Размерность энкодера: (d_model)
    9) Dropout: (dropout)
    
    
    


In [11]:
model = tst.PatchTST(M=len(companies),L=104,P=56, S=1, num_heads=8, dim_feedforward=2048, 
                 num_layers=4, d_model=512, dropout=0.1).to(device)

In [ ]:
# Train the model
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=3, verbose=True)

epochs = 50
early_stop_count = 0
min_val_loss = float('inf')

train_loss_per_epoch = []
val_loss_per_epoch = []
train_winrate = []
val_winrate = []



pbar = trange(epochs, desc="Epoch:")
since = time.time()

for epoch in pbar:
    model.train()
    train_losses = 0
    train_win = 0
    for batch in tqdm(train_loader, leave=False, desc="Train iter:"):
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        train_losses += loss.item()
        optimizer.step()
        train_loss = train_losses/len(train_loader)
        train_win += tst.winrate(y_batch, outputs)
    train_loss_per_epoch.append(train_loss)    
    train_winrate.append(train_win / len(train_loader))
    
    # Validation
    model.eval()
    val_losses = 0
    val_win = 0
    with torch.no_grad():
        for batch in tqdm(val_loader, leave=False, desc="Val iter:"):
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses += loss.item()
            val_win += tst.winrate(y_batch, outputs)
    val_loss = val_losses/len(val_loader)
    val_loss_per_epoch.append(val_loss)
    val_winrate.append(val_win / len(val_loader))
    
#     scheduler.step(val_loss)
#     if val_loss < 0.00012:
#         torch.save(model.state_dict(), f'МОДЕЛИ/Веса для TSMixer(5 blocks) val_loss = {val_loss}' )
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1

    if early_stop_count >= 15:
        print("Early stopping!")
        break
    print(f"Epoch {epoch + 1}","/",f"{epochs},Train Loss: {train_loss:.6f} Validation Loss: {val_loss:.6f}")
time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))    

Epoch::   0%|          | 0/50 [00:00<?, ?it/s]

Train iter::   0%|          | 0/347 [00:00<?, ?it/s]

Val iter::   0%|          | 0/95 [00:00<?, ?it/s]

Epoch 1 / 50,Train Loss: 0.362949 Validation Loss: 0.184090


Train iter::   0%|          | 0/347 [00:00<?, ?it/s]

...

Epoch 7 / 50,Train Loss: 0.032998 Validation Loss: 0.054233


Train iter::   0%|          | 0/347 [00:00<?, ?it/s]

Val iter::   0%|          | 0/95 [00:00<?, ?it/s]

Epoch 8 / 50,Train Loss: 0.037462 Validation Loss: 0.026367


Train iter::   0%|          | 0/347 [00:00<?, ?it/s]

In [ ]:
# Evaluation
model.eval()
predictions1 = []
with torch.no_grad():
    for batch in val_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        predictions1.extend(outputs.squeeze().tolist())
predictions1 = np.array(predictions1)

In [ ]:
tst.winrate(y_val, torch.Tensor(predictions1))

In [ ]:
plt.plot(train_loss_per_epoch)
plt.plot(val_loss_per_epoch)
plt.legend(['train','val'])

In [ ]:
plt.plot(train_winrate)
plt.plot(val_winrate)
plt.legend(['Train','Val'])

In [ ]:
plt.plot(y_val[:,0][:100])
plt.plot(torch.Tensor(predictions1)[:,0][:100])
plt.legend(['Real','Pred'])

In [ ]:
torch.Tensor(predictions1)[:,0]